# 3D Segmentation of Brain Tumor

This document is to test the methodology and refine it for execution in FYP B.


## Loading the Dataset
The dataset used is the BraTS 2020 Dataset

In [12]:
from monai.data import Dataset, CacheDataset,PersistentDataset
from glob import glob
from os import path, listdir
import albumentations as A
from itkwidgets import view

from monai.transforms import (
    LoadImaged,
    SpatialPadd,
    EnsureChannelFirstd,
    Compose,
    NormalizeIntensityd,
    Orientationd,
    Spacingd,
    RandSpatialCropd,
    RandFlipd,
    MapTransform
)

import monai
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [39]:
%pip install monai==0.9.0

  Using cached monai-0.9.0-202206131636-py3-none-any.whl (939 kB)
  Attempting uninstall: monai
    Found existing installation: monai 0.4.0
    Uninstalling monai-0.4.0:
      Successfully uninstalled monai-0.4.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
base_path = r"Datasets\2020\MICCAI_BraTS2020_TrainingData"
# Base Path to the training Dataset

In [18]:
# One Hot Encoding for Tumor as 3 is skipped in the segmentation masks. Moving 4 to 3.
class ConvertLabelsIntoOneHotd(MapTransform):
    def __call__(self, data):
        data_dict = dict(data)
        for key in self.keys:
            one_hot_encode_array = [
                data_dict[key] == 1,  # Non Enhancing Tumor Core
                data_dict[key] == 2,  # Edema Core
                data_dict[key] == 4,  # Enhancing Tumor Core
            ]
            data_dict[key] = np.stack(one_hot_encode_array, axis=0).astype(np.float32)
        return data_dict


In [25]:
cases = [path.join(base_path, x) for x in listdir(base_path)]
# Appends all cases to a list

mri_training_images = [
    {
        "image": [
            path.join(case, f"{path.split(case)[-1]}_t1ce.nii.gz"),
            path.join(case, f"{path.split(case)[-1]}_t2.nii.gz"),
            path.join(case, f"{path.split(case)[-1]}_flair.nii.gz"),
        ],
        "seg": path.join(case, f"{path.split(case)[-1]}_seg.nii.gz"),
    }
    for case in cases[:100]  #! Only 100 cases taken as test slice
]
# Extract the path to the training nifti files of each case. Chosen are t1ce, t2 and flair. Segmentation is also extracted.

# TODO: Research more about the following transforms as well as importance of transforms for MRI data.
transform_images = Compose(
    [
        LoadImaged(keys=["image", "seg"]),  # Loads the images and segmentation
        EnsureChannelFirstd(
            keys=["image", "seg"]
        ),  # Ensures that the images are in the correct format, ie channel is first dimension.
        ConvertLabelsIntoOneHotd(
            keys=["seg"]
        ),  # Converts the segmentation into one hot encoding
        # SpatialPadd(
        #     keys=["image", "seg"], spatial_size=(256, 256, 155)
        # ),  # Ensures that image dimensions are divisible by 32 for U-Net
        # NormalizeIntensityd(
        #     keys="image", nonzero=True, channel_wise=True
        # ),  # Normalise Intensity of the images
        # Orientationd(keys=["image", "seg"], axcodes="RAS"),
        # Spacingd(
        #     keys=["image", "seg"],
        #     pixdim=(1.0, 1.0, 1.0),
        #     mode=("bilinear", "nearest"),
        # ),  # Ensure spacing of voxels are uniform among scans
        # RandFlipd(keys=["image", "seg"], prob=0.5, spatial_axis=0),
        # RandFlipd(keys=["image", "seg"], prob=0.5, spatial_axis=1),
        # RandFlipd(
        #     keys=["image", "seg"], prob=0.5, spatial_axis=2
        # ),  # Three Random flips along the three axes
    ]
)
# Defines the transformations to be applied to the training data.

persistent_dataset_path = (
    r"./cache"  # Path of the persistent dataset cache, to speed up training process
)

dataset_training = PersistentDataset(
    mri_training_images, transform=transform_images, cache_dir=persistent_dataset_path
)  # Preparation of the training dataset


In [26]:
dataset_training[0]

RuntimeError: applying transform <monai.transforms.spatial.dictionary.Orientationd object at 0x00000180CC3B37C0>

In [9]:
slice[slice==4]=3

In [21]:
np.unique(slice)

array([0., 1., 2., 4.], dtype=float32)

In [14]:
dataset_training.__len__()

100

In [15]:
np.logical_or(5==5,4==2)

True

T1: T1-weighted, native image, sagittal or axial 2D acquisitions, with 1–6 mm slice thickness.

T1c: T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.

T2: T2-weighted image, axial 2D acquisition, with 2–6 mm slice thickness.

FLAIR: T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2–6 mm slice thickness.

So for training T1c and FLAIR will be used.

1. Non-Enhancing
2. Edema
3. Skipped
4. Enhancing